# O projeto é uma análise de dados que deve conter as seguintes etapas:

•	Carregar e ler arquivos .csv, xlsx ou de um banco de dados;

•	Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo);

•	Realizar a codificação das variáveis categóricas conforme os valores das variáveis (label encoder, one hot encoder ou target encoder);

•	Normalizar as variáveis;

•	Avaliar a necessidade de realizar o balanceamento da variável alvo;

•	Tratar variáveis com alta correlação;

•	Realizar a seleção de variáveis;

•	Aplicar um modelo de regressão ou classificação utilizando uma técnica de hiperparametrização automática;

•	Aplicar uma medida de avaliação do modelo.

•	Gerar uma visualização para o resultado conforme os tipos abaixo:
    - Agrupamento: gerar a visualização Scatter;
    - Classificação: gerar a matriz de confusão; 
    - Regressão: gerar a visualização da linha de saída.


  Importando Bibliotecas

In [14]:
import pandas as pd # O 'as' é um alias, um apelido. Ao invés de escrever 'pandas', escrevemos 'pd'
import numpy as np

from sklearn.cluster import MiniBatchKMeans
from imblearn.over_sampling import SMOTE, RandomOverSampler, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.under_sampling import ClusterCentroids, CondensedNearestNeighbour, EditedNearestNeighbours
from imblearn.under_sampling import RepeatedEditedNearestNeighbours, TomekLinks, AllKNN, InstanceHardnessThreshold
from imblearn.under_sampling import NearMiss, NeighbourhoodCleaningRule, OneSidedSelection, RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve

# 1. Carregar o arquivo .csv usando o pd.read_csv()

In [15]:
df = pd.read_csv('Pokemon.csv') # Lê o arquivo .csv e armazena em um DataFrame
df.head() #  Traz os 5 primeiros registros do DataFrame

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 2. Avaliar a necessidade da transformação da variável em outra escala (agrupar idade por faixas, por exemplo)

Não vimos necessidade de escalar as variáveis, pois o dataset é pequeno e autoexplicativo.

In [16]:
df.drop(['#'], axis=1, inplace=True) # Remove a coluna '#'; o axis 1 indica que é uma coluna, se fosse 0, indicaria que é uma linha; inplace=True indica que a alteração é feita no próprio DataFrame
colunas = ['nome', 'tipo_1', 'tipo_2', 'total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade', 'geracao', 'lendario'] # traduzindo as colunas e renomeando de uma forma mais amigável para o modelo, tirando espaços e maiúsculas
df.columns = colunas # Renomeia as colunas do DataFrame
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


# 3. Realizar a codificação das variáveis categóricas de acordo com os valores das variáveis (label encoder, one hot encoder ou target encoder)

In [17]:
# Label Encoder

from sklearn.preprocessing import LabelEncoder # Importa a classe LabelEncoder do módulo preprocessing da biblioteca sklearn

le = LabelEncoder() # O LabelEncoder faz sentido quando a variável categórica é ordinal, ou seja, tem uma ordem natural. O pokemon tem mais 'peso' por ser lendário.

df['lendario'] = le.fit_transform(df['lendario']) # Transforma os valores da coluna lendario em valores numéricos [0-1]
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,0


# Normalizar as variáveis

In [18]:
# Antes, uma análise descritiva do DataFrame, é bom para observar valores discrepantes, outliers, e ter uma noção da distribuição dos dados.
df.describe()

,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
count,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.00000
mean,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375,0.08125
std,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129,0.27339
min,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000,0.00000
25%,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000,0.00000
50%,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000,0.00000
75%,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000,0.00000
max,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000,1.00000


In [19]:
# Antes, vamos tratar os NaN da coluna do tipo_2, pois muitos pokemons não tem 2 tipos de categorias.

df['tipo_2'].fillna('None', inplace=True) # Preenche os valores NaN da coluna tipo_2 com 'None'
df.isna().sum() # Verifica se há valores NaN no DataFrame

nome          0
tipo_1        0
tipo_2        0
total         0
hp            0
ataque        0
defesa        0
sp_atq        0
sp_def        0
velocidade    0
geracao       0
lendario      0
dtype: int64

In [20]:
df.head()

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0


In [21]:
from sklearn.preprocessing import StandardScaler # Importa a classe StandardScaler do módulo preprocessing da biblioteca sklearn

# O standardScaler é uma técnica de normalização que transforma os dados para que a média seja 0 e o desvio padrão seja 1.
# Se o valor da variável for menor que a média, o valor transformado será negativo. Se for maior, será positivo. Quanto maior o número, mais próximo, ou até maior que 1 ele será.

scaler = StandardScaler() # Instancia o StandardScaler
# Vamos usar o Scaler, pois ele é mais sensível a outliers, o que é comum em pokemons lendários. 

# Para manter o DataFrame original, vamos criar um DataFrame com os valores normalizados

df_norm = df.copy() # Cria uma cópia do DataFrame
df_norm[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']] = scaler.fit_transform(df_norm[['total', 'hp', 'ataque', 'defesa', 'sp_atq', 'sp_def', 'velocidade']]) # Normaliza as colunas int64 e float64

df_norm.head()


,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,-0.976765,-0.950626,-0.924906,-0.797154,-0.239130,-0.248189,-0.801503,1,0
1,Ivysaur,Grass,Poison,-0.251088,-0.362822,-0.524130,-0.347917,0.219560,0.291156,-0.285015,1,0
2,Venusaur,Grass,Poison,0.749845,0.420917,0.092448,0.293849,0.831146,1.010283,0.403635,1,0
3,VenusaurMega Venusaur,Grass,Poison,1.583957,0.420917,0.647369,1.577381,1.503891,1.729409,0.403635,1,0
4,Charmander,Fire,None,-1.051836,-1.185748,-0.832419,-0.989683,-0.392027,-0.787533,-0.112853,1,0


In [22]:
df.head() # Como visto, manteve o df original.

,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,0
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,0
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,0
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,0
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0


Analisando a discrepância dos dados usando porcentagem, assim teremos um melhor insight sobre a distribuição dos dados e o que fazer com eles.

In [23]:
# Contagem de Pokémon lendários
num_lendario = df["lendario"].sum()
print(num_lendario)

# Contagem total de Pokémon
total_pokemon = len(df)
print(total_pokemon)

# Proporção de Pokémon lendários.
proporcao_lendarios = num_lendario / total_pokemon

print(f"Proporção de Pokémon lendários: {proporcao_lendarios:.2%}")

65
800
Proporção de Pokémon lendários: 8.12%


Deletando as colunas do tipo string para que possamos aplicar o SMOTE.

In [24]:
colunas_str = df_norm.select_dtypes(include=['object']).columns # Pegando todas as colunas do tipo "Object" e armazenando em uma lista

df_norm.drop(columns=colunas_str, inplace=True) # Passando a lista que pegamos acima, para que seja removido todas as colunas que nela pertence
df_norm.head()

,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
0,-0.976765,-0.950626,-0.924906,-0.797154,-0.239130,-0.248189,-0.801503,1,0
1,-0.251088,-0.362822,-0.524130,-0.347917,0.219560,0.291156,-0.285015,1,0
2,0.749845,0.420917,0.092448,0.293849,0.831146,1.010283,0.403635,1,0
3,1.583957,0.420917,0.647369,1.577381,1.503891,1.729409,0.403635,1,0
4,-1.051836,-1.185748,-0.832419,-0.989683,-0.392027,-0.787533,-0.112853,1,0


SMOTE : Criou cópias dos Pokémon lendários para equilibrar as classes.

In [25]:
# Separando as features (X) e o target (y)

X = df_norm.drop(columns=["lendario"])
y = df_norm["lendario"]

In [26]:
# Separação dos dados de treino (80%) e teste (20%)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # o test_size=0.2 indica que 20% dos dados serão usados para teste


In [27]:
#Normalizando dos dados de treino e teste

min_max = MinMaxScaler()
min_max.fit(X_train)
X_train_norm = min_max.transform(X_train)
X_test_norm = min_max.transform(X_test)

In [28]:
# Aplicando o SMOTE para oversampling

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train_norm, y_train)
X_smote.shape


(1170, 8)

In [29]:
y_smote.value_counts()

1    585
0    585
Name: lendario, dtype: int64

SVM

In [30]:
modelo_svm = SVC(kernel='rbf', random_state=42)
modelo_svm.fit(X_smote, y_smote)
y_pred = modelo_svm.predict(X_test_norm)

In [31]:
print('Acurácia:', accuracy_score(y_test, y_pred))
print('Revocação:', recall_score(y_test, y_pred, average='weighted'))
print('Precisão:', precision_score(y_test, y_pred,average='weighted'))

Acurácia: 0.9375
Revocação: 0.9375
Precisão: 0.96875


knn

In [32]:
modelo_knn = KNeighborsClassifier(n_neighbors=3)
modelo_knn.fit(X_smote, y_smote)
y_pred = modelo_knn.predict(X_test_norm)

In [33]:
print('Acurácia:', accuracy_score(y_test, y_pred))
print('Revocação:', recall_score(y_test, y_pred, average='weighted'))
print('Precisão:', precision_score(y_test, y_pred,average='weighted'))

Acurácia: 0.93125
Revocação: 0.93125
Precisão: 0.9604563269876818


Undersampling: ReduzaREduzindo aleatoriamente o número de exemplos da classe majoritária (não lendários)

In [34]:
# Contagem de exemplos por classe
contagem_por_classe = df["lendario"].value_counts()
contagem_por_classe

0    735
1     65
Name: lendario, dtype: int64

In [35]:
# Defenindo o número máximo de exemplos por classe (no seu caso, 500)
n_maximo_por_classe = 500

In [36]:
# Filtrando as classes com mais exemplos do que o limite
classes_maior_que_limite = contagem_por_classe[contagem_por_classe > n_maximo_por_classe].index

In [37]:
# Faazendo uma seleção aleatória de exemplos das classes maiores que o limite
exemplos_undersampled = pd.concat([
    df[df["lendario"] == classe].sample(n=n_maximo_por_classe, replace=False)
    for classe in classes_maior_que_limite
])

In [38]:
# Mantendo os exemplos das classes menores que o limite
exemplos_undersampled = pd.concat([
    exemplos_undersampled,
    df[df["lendario"].isin(classes_maior_que_limite) == False]
])

In [39]:
# Agora exemplos_undersampled contém o DataFrame com a classe "lendário" balanceada
exemplos_undersampled


,nome,tipo_1,tipo_2,total,hp,ataque,defesa,sp_atq,sp_def,velocidade,geracao,lendario
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,0
524,Mamoswine,Ice,Ground,530,110,130,80,70,60,80,4,0
750,AegislashBlade Forme,Steel,Ghost,520,60,150,50,150,50,60,6,0
716,MeloettaPirouette Forme,Normal,Fighting,600,100,128,90,77,77,128,5,0
110,Exeggcute,Grass,Psychic,325,60,40,80,60,45,40,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,1
796,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,1
797,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,1
798,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,1


In [40]:
#Randon Sampling gerando numeros mais proximos utilizando uma matriz de covariancia
randon_sampler = RandomOverSampler(random_state=42)
X_random, y_random = randon_sampler.fit_resample(X,y)
X_random.shape

(1470, 8)

In [41]:
y_random.shape

(1470,)

In [42]:
X_random.value_counts()

total      hp         ataque     defesa     sp_atq     sp_def     velocidade  geracao
 2.042718   2.223516   1.603065   0.678908   1.779105   0.938370   1.057853   6          27
 1.208607   0.381730   1.109803  -0.123299   1.595629   0.291156   1.471044   5          23
 2.209541  -0.754692   2.497104   1.160233   2.665905   1.369846   1.436611   6          20
 2.042718   1.204656   1.263947   1.481116   2.360112   1.010283   0.747961   4          19
 1.208607   0.812786   0.184935   0.839350   0.678249   1.909191   0.575798   1          17
                                                                                         ..
-0.584733   0.029047  -0.524130  -0.219564  -0.544924  -0.607752  -0.422745   1           1
-0.551369  -0.284448  -0.894077  -0.508359   0.005504  -0.643708   0.128175   6           1
-0.543027  -0.362822  -1.202366  -0.444182   0.678249  -0.427970  -0.457178   5           1
           -0.166887  -1.202366  -0.765065   1.595629  -0.427970  -1.317991   5       

In [43]:
y_random.value_counts()

0    735
1    735
Name: lendario, dtype: int64

In [44]:
rs = pd.DataFrame({"Nome": y.value_counts().index,
              "Qte.": y.value_counts().values,
              "% Total": round(y.value_counts(normalize=True) * 100,2).values})

In [45]:
rs

,Nome,Qte.,% Total
0,0,735,91.88
1,1,65,8.12


In [46]:
centroids_under =ClusterCentroids(estimator=MiniBatchKMeans(n_init=10, random_state=0), random_state=42)
X_centroid, y_centroid = centroids_under.fit_resample(X,y)
X_centroid.shape

C:\Users\vinissoissoisso\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1902: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 3072 or by setting the environment variable OMP_NUM_THREADS=3
  warnings.warn(


(130, 8)

In [47]:
y_centroid.value_counts()

0    65
1    65
Name: lendario, dtype: int64

In [48]:
df_norm.shape


(800, 9)